In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
data=pd.read_csv("fake_news.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
data.shape

(20800, 5)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [11]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
data=data.drop(['id'], axis=1)

In [13]:
data=data.fillna('')

In [15]:
data['content'] = data['author']+' '+data['title']+' '+data['text']

In [16]:
data=data.drop(['title','author','text'],axis=1)

In [17]:
data.head()

,label,content
0,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,1,Consortiumnews.com Why the Truth Might Get You...
3,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,1,Howard Portnoy Iranian woman jailed for fictio...


In [18]:
data['content']=data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [19]:
data['content']=data['content'].str.replace('[\w\s]','')

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\saisagar\AppData\Local\Temp\ipykernel_9280\2016841428.py:1: SyntaxWarning: invalid escape sequence '\w'
  data['content']=data['content'].str.replace('[\w\s]','')


In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saisagar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
data['content']=data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [22]:
!pip install textblob

In [23]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data['content'].head()

0    darrell lucus house dem aide: didn’t even see ...
1    daniel j. flynn flynn: hillary clinton, big wo...
2    consortiumnews.com truth might get fired truth...
3    jessica purkiss 15 civilians killed single us ...
4    howard portnoy iranian woman jailed fictional ...
Name: content, dtype: object

In [24]:
x=data[['content']]
y=data['label']

In [25]:
from sklearn.model_selection import train_test_split

In [31]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=45,stratify=y)

In [33]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(14560, 1)
(14560,)
(6240, 1)
(6240,)


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [38]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}')
tfidf_vect.fit(x_train['content'])
xtrain_tfidf =  tfidf_vect.transform(x_train['content'])
xtest_tfidf =  tfidf_vect.transform(x_test['content'])

In [39]:
from sklearn import model_selection,linear_model,metrics

In [40]:
from sklearn.linear_model import PassiveAggressiveClassifier
pclf=PassiveAggressiveClassifier()
pclf.fit(xtrain_tfidf,y_train)
predictions=pclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      3116
           1       0.98      0.97      0.98      3124

    accuracy                           0.98      6240
   macro avg       0.98      0.98      0.98      6240
weighted avg       0.98      0.98      0.98      6240



In [57]:
print(metrics.confusion_matrix(y_test,predictions))

[[3050   66]
 [  80 3044]]


In [ ]:
from sklearn.neural_network import MLPClassifier
mlpclf=MLPClassifier(hidden_layer_sizes=(256,64,16),
                    activation='relu',
                    solver='adam')
mlpclf.fit(xtrain_tfidf,y_train)
predictions=mlpclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test,predictions))

In [ ]:
import pickle
pickle.dump(mlpclf,open("fakenews1.pkl",'wb'))